# 準備

## ディレクトリを移動

以下では，Googleドライブのマイドライブ直下にDNN_codeフォルダを置くことを仮定しています．必要に応じて，パスを変更してください．

In [1]:
import os
os.chdir('drive/My Drive/DNN_code/lesson_3/3_2_tf_languagemodel/')

FileNotFoundError: [Errno 2] No such file or directory: 'drive/My Drive/DNN_code/lesson_3/3_2_tf_languagemodel/'

# predict_word

In [1]:
import tensorflow as tf
import numpy as np
import re
import glob
import collections
import random
import pickle
import time
import datetime
import os

# logging levelを変更
tf.logging.set_verbosity(tf.logging.ERROR)

class Corpus:
    def __init__(self):
        self.unknown_word_symbol = "<???>" # 出現回数の少ない単語は未知語として定義しておく
        self.unknown_word_threshold = 3 # 未知語と定義する単語の出現回数の閾値
        self.corpus_file = "./corpus/**/*.txt"
        self.corpus_encoding = "utf-8"
        self.dictionary_filename = "./data_for_predict/word_dict.dic"
        self.chunk_size = 5
        self.load_dict()

        words = []
        for filename in glob.glob(self.corpus_file, recursive=True):
            with open(filename, "r", encoding=self.corpus_encoding) as f:

                # word breaking
                text = f.read()
                # 全ての文字を小文字に統一し、改行をスペースに変換
                text = text.lower().replace("\n", " ")
                # 特定の文字以外の文字を空文字に置換する
                text = re.sub(r"[^a-z '\-]", "", text)
                # 複数のスペースはスペース一文字に変換
                text = re.sub(r"[ ]+", " ", text)

                # 前処理： '-' で始まる単語は無視する
                words = [ word for word in text.split() if not word.startswith("-")]


        self.data_n = len(words) - self.chunk_size
        self.data = self.seq_to_matrix(words)

    def prepare_data(self):
        """
        訓練データとテストデータを準備する。
        data_n = ( text データの総単語数 ) - chunk_size
        input: (data_n, chunk_size, vocabulary_size)
        output: (data_n, vocabulary_size)
        """

        # 入力と出力の次元テンソルを準備
        all_input = np.zeros([self.chunk_size, self.vocabulary_size, self.data_n])
        all_output = np.zeros([self.vocabulary_size, self.data_n])

        # 準備したテンソルに、コーパスの one-hot 表現(self.data) のデータを埋めていく
        # i 番目から ( i + chunk_size - 1 ) 番目までの単語が１組の入力となる
        # このときの出力は ( i + chunk_size ) 番目の単語
        for i in range(self.data_n):
            all_output[:, i] = self.data[:, i + self.chunk_size] # (i + chunk_size) 番目の単語の one-hot ベクトル
            for j in range(self.chunk_size):
                all_input[j, :, i] = self.data[:, i + self.chunk_size - j - 1]

        # 後に使うデータ形式に合わせるために転置を取る
        all_input = all_input.transpose([2, 0, 1])
        all_output = all_output.transpose()

        # 訓練データ：テストデータを 4 : 1 に分割する
        training_num = ( self.data_n * 4 ) // 5
        return all_input[:training_num], all_output[:training_num], all_input[training_num:], all_output[training_num:]


    def build_dict(self):
        # コーパス全体を見て、単語の出現回数をカウントする
        counter = collections.Counter()
        for filename in glob.glob(self.corpus_file, recursive=True):
            with open(filename, "r", encoding=self.corpus_encoding) as f:

                # word breaking
                text = f.read()
                # 全ての文字を小文字に統一し、改行をスペースに変換
                text = text.lower().replace("\n", " ")
                # 特定の文字以外の文字を空文字に置換する
                text = re.sub(r"[^a-z '\-]", "", text)
                # 複数のスペースはスペース一文字に変換
                text = re.sub(r"[ ]+", " ", text)

                # 前処理： '-' で始まる単語は無視する
                words = [word for word in text.split() if not word.startswith("-")]

                counter.update(words)

        # 出現頻度の低い単語を一つの記号にまとめる
        word_id = 0
        dictionary = {}
        for word, count in counter.items():
            if count <= self.unknown_word_threshold:
                continue

            dictionary[word] = word_id
            word_id += 1
        dictionary[self.unknown_word_symbol] = word_id

        print("総単語数：", len(dictionary))

        # 辞書を pickle を使って保存しておく
        with open(self.dictionary_filename, "wb") as f:
            pickle.dump(dictionary, f)
            print("Dictionary is saved to", self.dictionary_filename)

        self.dictionary = dictionary

        print(self.dictionary)

    def load_dict(self):
        with open(self.dictionary_filename, "rb") as f:
            self.dictionary = pickle.load(f)
            self.vocabulary_size = len(self.dictionary)
            self.input_layer_size = len(self.dictionary)
            self.output_layer_size = len(self.dictionary)
            print("総単語数: ", self.input_layer_size)

    def get_word_id(self, word):
        # print(word)
        # print(self.dictionary)
        # print(self.unknown_word_symbol)
        # print(self.dictionary[self.unknown_word_symbol])
        # print(self.dictionary.get(word, self.dictionary[self.unknown_word_symbol]))
        return self.dictionary.get(word, self.dictionary[self.unknown_word_symbol])

    # 入力された単語を one-hot ベクトルにする
    def to_one_hot(self, word):
        index = self.get_word_id(word)
        data = np.zeros(self.vocabulary_size)
        data[index] = 1
        return data

    def seq_to_matrix(self, seq):
        print(seq)
        data = np.array([self.to_one_hot(word) for word in seq]) # (data_n, vocabulary_size)
        return data.transpose() # (vocabulary_size, data_n)

class Language:
    """
    input layer: self.vocabulary_size
    hidden layer: rnn_size = 30
    output layer: self.vocabulary_size
    """

    def __init__(self):
        self.corpus = Corpus()
        self.dictionary = self.corpus.dictionary
        self.vocabulary_size = len(self.dictionary) # 単語数
        self.input_layer_size = self.vocabulary_size # 入力層の数
        self.hidden_layer_size = 30 # 隠れ層の RNN ユニットの数
        self.output_layer_size = self.vocabulary_size # 出力層の数
        self.batch_size = 128 # バッチサイズ
        self.chunk_size = 5 # 展開するシーケンスの数。c_0, c_1, ..., c_(chunk_size - 1) を入力し、c_(chunk_size) 番目の単語の確率が出力される。
        self.learning_rate = 0.005 # 学習率
        self.epochs = 1000 # 学習するエポック数
        self.forget_bias = 1.0 # LSTM における忘却ゲートのバイアス
        self.model_filename = "./data_for_predict/predict_model.ckpt"
        self.unknown_word_symbol = self.corpus.unknown_word_symbol

    def inference(self, input_data, initial_state):
        """
        :param input_data: (batch_size, chunk_size, vocabulary_size) 次元のテンソル
        :param initial_state: (batch_size, hidden_layer_size) 次元の行列
        :return:
        """
        # 重みとバイアスの初期化
        hidden_w = tf.Variable(tf.truncated_normal([self.input_layer_size, self.hidden_layer_size], stddev=0.01))
        hidden_b = tf.Variable(tf.ones([self.hidden_layer_size]))
        output_w = tf.Variable(tf.truncated_normal([self.hidden_layer_size, self.output_layer_size], stddev=0.01))
        output_b = tf.Variable(tf.ones([self.output_layer_size]))

        # BasicLSTMCell, BasicRNNCell は (batch_size, hidden_layer_size) が chunk_size 数ぶんつながったリストを入力とする。
        # 現時点での入力データは (batch_size, chunk_size, input_layer_size) という３次元のテンソルなので
        # tf.transpose や tf.reshape などを駆使してテンソルのサイズを調整する。

        input_data = tf.transpose(input_data, [1, 0, 2]) # 転置。(chunk_size, batch_size, vocabulary_size)
        input_data = tf.reshape(input_data, [-1, self.input_layer_size]) # 変形。(chunk_size * batch_size, input_layer_size)
        input_data = tf.matmul(input_data, hidden_w) + hidden_b # 重みWとバイアスBを適用。 (chunk_size, batch_size, hidden_layer_size)
        input_data = tf.split(input_data, self.chunk_size, 0) # リストに分割。chunk_size * (batch_size, hidden_layer_size)

        # RNN のセルを定義する。RNN Cell の他に LSTM のセルや GRU のセルなどが利用できる。
        with tf.variable_scope('auto_reuse', reuse=tf.AUTO_REUSE):
            cell = tf.nn.rnn_cell.BasicRNNCell(self.hidden_layer_size)
            outputs, states = tf.nn.static_rnn(cell, input_data, initial_state=initial_state)
        
        # 最後に隠れ層から出力層につながる重みとバイアスを処理する
        # 最終的に softmax 関数で処理し、確率として解釈される。
        # softmax 関数はこの関数の外で定義する。
        output = tf.matmul(outputs[-1], output_w) + output_b

        return output

    def loss(self, logits, labels):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))

        return cost

    def training(self, cost):
        # 今回は最適化手法として Adam を選択する。
        # ここの AdamOptimizer の部分を変えることで、Adagrad, Adadelta などの他の最適化手法を選択することができる
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(cost)

        return optimizer

    def train(self):
        # 変数などの用意
        input_data = tf.placeholder("float", [None, self.chunk_size, self.input_layer_size])
        actual_labels = tf.placeholder("float", [None, self.output_layer_size])
        initial_state = tf.placeholder("float", [None, self.hidden_layer_size])

        prediction = self.inference(input_data, initial_state)
        cost = self.loss(prediction, actual_labels)
        optimizer = self.training(cost)
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(actual_labels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

        # TensorBoard で可視化するため、クロスエントロピーをサマリーに追加
        tf.summary.scalar("Cross entropy: ", cost)
        summary = tf.summary.merge_all()

        # 訓練・テストデータの用意
        # corpus = Corpus()
        trX, trY, teX, teY = self.corpus.prepare_data()
        training_num = trX.shape[0]

        # ログを保存するためのディレクトリ
        timestamp = time.time()
        dirname = datetime.datetime.fromtimestamp(timestamp).strftime("%Y%m%d%H%M%S")

        # ここから実際に学習を走らせる
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            summary_writer = tf.summary.FileWriter("./log/" + dirname, sess.graph)

            # エポックを回す
            for epoch in range(self.epochs):
                step = 0
                epoch_loss = 0
                epoch_acc = 0

                # 訓練データをバッチサイズごとに分けて学習させる (= optimizer を走らせる)
                # エポックごとの損失関数の合計値や（訓練データに対する）精度も計算しておく
                while (step + 1) * self.batch_size < training_num:
                    start_idx = step * self.batch_size
                    end_idx = (step + 1) * self.batch_size

                    batch_xs = trX[start_idx:end_idx, :, :]
                    batch_ys = trY[start_idx:end_idx, :]

                    _, c, a = sess.run([optimizer, cost, accuracy],
                                       feed_dict={input_data: batch_xs,
                                                  actual_labels: batch_ys,
                                                  initial_state: np.zeros([self.batch_size, self.hidden_layer_size])
                                                  }
                                       )
                    epoch_loss += c
                    epoch_acc += a
                    step += 1

                # コンソールに損失関数の値や精度を出力しておく
                print("Epoch", epoch, "completed ouf of", self.epochs, "-- loss:", epoch_loss, " -- accuracy:",
                      epoch_acc / step)

                # Epochが終わるごとにTensorBoard用に値を保存
                summary_str = sess.run(summary, feed_dict={input_data: trX,
                                                           actual_labels: trY,
                                                           initial_state: np.zeros(
                                                               [trX.shape[0],
                                                                self.hidden_layer_size]
                                                           )
                                                           }
                                       )
                summary_writer.add_summary(summary_str, epoch)
                summary_writer.flush()

            # 学習したモデルも保存しておく
            saver = tf.train.Saver()
            saver.save(sess, self.model_filename)

            # 最後にテストデータでの精度を計算して表示する
            a = sess.run(accuracy, feed_dict={input_data: teX, actual_labels: teY,
                                              initial_state: np.zeros([teX.shape[0], self.hidden_layer_size])})
            print("Accuracy on test:", a)


    def predict(self, seq):
        """
        文章を入力したときに次に来る単語を予測する
        :param seq: 予測したい単語の直前の文字列。chunk_size 以上の単語数が必要。
        :return:
        """

        # 最初に復元したい変数をすべて定義してしまいます
        tf.reset_default_graph()
        input_data = tf.placeholder("float", [None, self.chunk_size, self.input_layer_size])
        initial_state = tf.placeholder("float", [None, self.hidden_layer_size])
        prediction = tf.nn.softmax(self.inference(input_data, initial_state))
        predicted_labels = tf.argmax(prediction, 1)

        # 入力データの作成
        # seq を one-hot 表現に変換する。
        words = [word for word in seq.split() if not word.startswith("-")]
        x = np.zeros([1, self.chunk_size, self.input_layer_size])
        for i in range(self.chunk_size):
            word = seq[len(words) - self.chunk_size + i]
            index = self.dictionary.get(word, self.dictionary[self.unknown_word_symbol])
            x[0][i][index] = 1
        feed_dict = {
            input_data: x, # (1, chunk_size, vocabulary_size)
            initial_state: np.zeros([1, self.hidden_layer_size])
        }

        # tf.Session()を用意
        with tf.Session() as sess:
            # 保存したモデルをロードする。ロード前にすべての変数を用意しておく必要がある。
            saver = tf.train.Saver()
            saver.restore(sess, self.model_filename)

            # ロードしたモデルを使って予測結果を計算
            u, v = sess.run([prediction, predicted_labels], feed_dict=feed_dict)

            keys = list(self.dictionary.keys())


            # コンソールに文字ごとの確率を表示
            for i in range(self.vocabulary_size):
                c = self.unknown_word_symbol if i == (self.vocabulary_size - 1) else keys[i]
                print(c, ":", u[0][i])

            print("Prediction:", seq + " " + ("<???>" if v[0] == (self.vocabulary_size - 1) else keys[v[0]]))

        return u[0]


def build_dict():
    cp = Corpus()
    cp.build_dict()

if __name__ == "__main__":
    build_dict()

    ln = Language()

    # 学習するときに呼び出す
    ln.train()

    # 保存したモデルを使って単語の予測をする
    ln.predict("some of them looks like")


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

総単語数:  9826
['welcome', 'back', 'to', 'our', 'show', 'ok', 'this', 'is', 'maria', 'and', 'i', "don't", 'know', 'your', 'last', 'name', 'polk', 'oh', "that's", 'good', 'from', 'polkton', 'maria', 'polk', 'ok', 'and', 'where', 'did', 'you', 'grow', 'up', 'were', 'you', 'born', 'in', 'this', 'area', 'here', 'i', 'was', 'born', 'in', 'cottonville', 'right', 'outside', 'of', 'norwood', 'in', 'stanley', 'county', 'oh', 'ok', 'when', 'you', 'were', 'little', 'did', 'your', 'mom', 'read', 'you', 'books', 'or', 'did', 'somebody', 'read', 'you', 'books', 'in', 'your', 'house', 'my', 'mama', 'read', 'to', 'me', 'and', 'my', 'sister', 'uh-huh', 'what', 'kind', 'of', 'books', 'were', 'your', 'favorites', 'do', 'you', 'remember', 'any', 'that', 'were', 'maybe', 'a', 'real', 'favorite', 'of', 'yours', 'mm', 'i', 'kind', 'of', 'liked', 'them', 'all', 'um', 'i', "didn't", 'really', 'have', 'a', 'favorite', 'no', 'any', 'particular', 'types', 'like', 'ones', 'with', 'pictures', 'or', 'can', 'you', 'thin

['welcome', 'back', 'to', 'our', 'show', 'ok', 'this', 'is', 'maria', 'and', 'i', "don't", 'know', 'your', 'last', 'name', 'polk', 'oh', "that's", 'good', 'from', 'polkton', 'maria', 'polk', 'ok', 'and', 'where', 'did', 'you', 'grow', 'up', 'were', 'you', 'born', 'in', 'this', 'area', 'here', 'i', 'was', 'born', 'in', 'cottonville', 'right', 'outside', 'of', 'norwood', 'in', 'stanley', 'county', 'oh', 'ok', 'when', 'you', 'were', 'little', 'did', 'your', 'mom', 'read', 'you', 'books', 'or', 'did', 'somebody', 'read', 'you', 'books', 'in', 'your', 'house', 'my', 'mama', 'read', 'to', 'me', 'and', 'my', 'sister', 'uh-huh', 'what', 'kind', 'of', 'books', 'were', 'your', 'favorites', 'do', 'you', 'remember', 'any', 'that', 'were', 'maybe', 'a', 'real', 'favorite', 'of', 'yours', 'mm', 'i', 'kind', 'of', 'liked', 'them', 'all', 'um', 'i', "didn't", 'really', 'have', 'a', 'favorite', 'no', 'any', 'particular', 'types', 'like', 'ones', 'with', 'pictures', 'or', 'can', 'you', 'think', 'of', 'a

Epoch 18 completed ouf of 1000 -- loss: 14.032709121704102  -- accuracy: 0.052083333333333336
Epoch 19 completed ouf of 1000 -- loss: 14.027478694915771  -- accuracy: 0.052083333333333336
Epoch 20 completed ouf of 1000 -- loss: 14.024359226226807  -- accuracy: 0.052083333333333336
Epoch 21 completed ouf of 1000 -- loss: 14.02202844619751  -- accuracy: 0.052083333333333336
Epoch 22 completed ouf of 1000 -- loss: 14.019739151000977  -- accuracy: 0.052083333333333336
Epoch 23 completed ouf of 1000 -- loss: 14.017264366149902  -- accuracy: 0.052083333333333336
Epoch 24 completed ouf of 1000 -- loss: 14.014694213867188  -- accuracy: 0.052083333333333336
Epoch 25 completed ouf of 1000 -- loss: 14.012228012084961  -- accuracy: 0.052083333333333336
Epoch 26 completed ouf of 1000 -- loss: 14.010030746459961  -- accuracy: 0.052083333333333336
Epoch 27 completed ouf of 1000 -- loss: 14.008158206939697  -- accuracy: 0.052083333333333336
Epoch 28 completed ouf of 1000 -- loss: 14.006569862365723  -

Epoch 106 completed ouf of 1000 -- loss: 13.975253105163574  -- accuracy: 0.052083333333333336
Epoch 107 completed ouf of 1000 -- loss: 13.975132942199707  -- accuracy: 0.052083333333333336
Epoch 108 completed ouf of 1000 -- loss: 13.975016117095947  -- accuracy: 0.052083333333333336
Epoch 109 completed ouf of 1000 -- loss: 13.97490119934082  -- accuracy: 0.052083333333333336
Epoch 110 completed ouf of 1000 -- loss: 13.974789142608643  -- accuracy: 0.052083333333333336
Epoch 111 completed ouf of 1000 -- loss: 13.97467851638794  -- accuracy: 0.052083333333333336
Epoch 112 completed ouf of 1000 -- loss: 13.974569320678711  -- accuracy: 0.052083333333333336
Epoch 113 completed ouf of 1000 -- loss: 13.974462985992432  -- accuracy: 0.052083333333333336
Epoch 114 completed ouf of 1000 -- loss: 13.974358081817627  -- accuracy: 0.052083333333333336
Epoch 115 completed ouf of 1000 -- loss: 13.974255561828613  -- accuracy: 0.052083333333333336
Epoch 116 completed ouf of 1000 -- loss: 13.97415447

...

convicted : 1.1438661e-10
wiretaps : 1.0649728e-10
tokhtakhounov's : 1.0191254e-10
gougne : 1.14665555e-10
gailhaguet : 1.12152995e-10
comey : 9.651352e-11
corrupt : 1.0396026e-10
singled : 1.10235356e-10
soviet : 1.12467896e-10
accompanying : 1.0976916e-10
affidavit : 1.09726034e-10
descriptions : 1.13483896e-10
identities : 9.899997e-11
renewing : 1.053809e-10
excerpts : 9.464515e-11
mobster : 9.636599e-11
referring : 1.0495364e-10
familiarity : 1.00731756e-10
stunned : 1.0464241e-10
competitors : 9.7173686e-11
compete : 9.711144e-11
upjohn : 1.1248764e-10
cosmetic : 1.0739173e-10
jonathan : 9.6385115e-11
shareholder : 1.06308747e-10
ala : 1.12205835e-10
vanderbilt : 1.0668046e-10
johnson : 9.754769e-11
signing : 1.0234694e-10
tech : 9.696448e-11
t-shirts : 1.1033506e-10
scores : 1.0173833e-10
coach : 1.12148506e-10
cutcliffe : 9.9667254e-11
weighs : 1.0443923e-10
rex : 9.873069e-11
trophy : 1.08352584e-10
promotional : 1.02380916e-10
miami : 1.1097965e-10
dow : 1.0259125e-10
telerat

dalma : 1.02465314e-10
claws : 1.1611726e-10
hoped : 9.5589994e-11
jar : 1.07933246e-10
sums : 1.0160588e-10
versions : 1.191642e-10
wid : 1.0555229e-10
beads : 1.146824e-10
scuttled : 1.0693429e-10
reef : 1.0460371e-10
awaits : 9.964179e-11
passes : 1.13789506e-10
crewmembers : 9.985564e-11
swordfight : 1.0498407e-10
slices : 1.0523146e-10
maccus : 1.0509526e-10
bare : 1.13363395e-10
ninety-nine : 1.1460019e-10
jones' : 1.1519518e-10
cabin : 9.933968e-11
platform : 1.11969385e-10
romantic : 1.0768874e-10
commodore : 1.03390164e-10
hired : 1.1259109e-10
habits : 9.814958e-11
fires : 1.0218699e-10
heave : 1.0992965e-10
orderly : 1.11544565e-10
bo'sun : 1.05311176e-10
cannon : 1.0453449e-10
tentacle : 1.03445585e-10
compassion : 1.0617642e-10
wyvern : 1.0548024e-10
stab : 9.9108784e-11
unhappy : 9.809979e-11
whatsoever : 9.737518e-11
koleniko : 1.037962e-10
would've : 1.0198449e-10
elizabeth's : 1.02499315e-10
messenger : 1.08034366e-10
weapon : 9.485189e-11
tilted : 9.665851e-11
flung :

iphone : 9.7111805e-11
kmhr : 9.965699e-11
columbine : 1.0348348e-10
right-wing : 1.0302303e-10
boyle : 1.06019186e-10
httptinyurlcomcrgl : 1.0468654e-10
gov't : 1.08612876e-10
teaparty : 1.0222637e-10
frequency : 1.1055077e-10
nintendo : 1.3166344e-10
wii : 9.6118925e-11
tweets : 1.01581465e-10
album : 9.573579e-11
presidency : 1.02813e-10
teabagging : 9.685283e-11
webinar : 1.0089906e-10
homosexuals : 1.0562318e-10
haha : 1.0878662e-10
prevailing : 1.04348245e-10
thanx : 1.06368775e-10
marketers : 1.0410432e-10
ourselves : 9.944585e-11
graham : 1.1415296e-10
bandwidth : 1.07659374e-10
recipe : 1.0688739e-10
libraries : 1.0146722e-10
fe : 9.9142444e-11
diapers : 1.0409003e-10
joins : 9.566276e-11
conservatives : 1.06335926e-10
denver : 9.102283e-11
treats : 1.079001e-10
nursing : 9.380748e-11
nra : 1.0117733e-10
fox : 1.0476283e-10
tuition : 1.0257403e-10
mason : 1.095182e-10
trout : 1.0038461e-10
directories : 9.4165294e-11
gossip : 1.039777e-10
guerrilla : 1.07943536e-10
appstore : 

anastomosing : 1.00007364e-10
uplift : 1.0411901e-10
deformed : 1.101538e-10
terraces : 1.0720242e-10
variations : 1.2123022e-10
dendritic : 1.0602687e-10
isostatic : 1.0633349e-10
incision : 1.0505457e-10
chattahoochee : 1.0509305e-10
subsidence : 1.0557786e-10
asymmetry : 1.1130399e-10
geological : 1.0301124e-10
contour : 1.1974588e-10
basin : 1.04005284e-10
ria : 1.1741567e-10
elongated : 1.009959e-10
knee : 1.0493042e-10
maran : 1.0039494e-10
dumont : 1.0373525e-10
superimposed : 1.0075828e-10
kyoga : 9.8439236e-11
diagram : 1.0444799e-10
northeast : 1.1525496e-10
vi : 1.07237406e-10
stevens : 1.01010346e-10
baselevel : 1.04663174e-10
cm : 9.378976e-11
valley-floor : 9.799058e-11
threshold : 1.01029615e-10
resistant : 1.0655214e-10
bedrock : 1.0320476e-10
compressed : 1.0705265e-10
meanders : 1.04588546e-10
journalism : 9.135765e-11
grip : 9.472263e-11
stephanopoulos : 1.0219986e-10
chatterbox : 9.5740894e-11
clinton's : 9.093139e-11
documentary : 1.12747964e-10
apology : 1.1102728

genocide : 1.02378765e-10
lockheed : 1.0667883e-10
heston : 1.0167102e-10
filmmaking : 9.893711e-11
mentions : 1.0307905e-10
kayla : 1.0420901e-10
rolland : 1.1341508e-10
distortion : 1.1280001e-10
faked : 9.9900595e-11
heston's : 1.0791945e-10
assertion : 1.0574271e-10
revitalization : 1.0353698e-10
maya : 1.0776045e-10
guatemala : 1.0065705e-10
guambiano : 1.0232977e-10
mayans : 9.632593e-11
mayan : 1.00755786e-10
rhetoric : 8.5644e-11
pan-mayan : 1.09657824e-10
guatemalan : 1.0237818e-10
modernity : 1.0552571e-10
gow : 9.763387e-11
cosmovision : 9.8970894e-11
harmonic : 1.1153328e-10
harvester : 1.01195864e-10
ant : 1.1291152e-10
undetected : 1.0002452e-10
task-switching : 1.0618837e-10
antie : 9.8011516e-11
patrolling : 9.83546e-11
midden : 1.0699345e-10
nests : 1.156456e-10
forage : 1.1211022e-10
schafer : 1.0155744e-10
mound : 1.0248368e-10
sub-behaviors : 1.0423942e-10
patrollers : 1.0441812e-10
cuticular : 1.1028415e-10
foragers : 9.972659e-11
patroller : 1.0302932e-10
inactive